In [2]:
import mysql.connector
from tabulate import tabulate
import pandas as pd
import math
import sys


In [4]:
HOST = "147.46.15.238"
PORT = "7000"
USER = "DS2024_0022"
PASSWD = "DS2024_0022"
DB = "DS_proj_15"

connection = mysql.connector.connect(
    host=HOST,
    port=7000,
    user=USER,
    passwd=PASSWD,
    db=DB,
    autocommit=True  # to create table permanently
)

cur = connection.cursor(dictionary=True)


In [5]:
def get_output(query):
    cur.execute(query)
    out = cur.fetchall()
    df = pd.DataFrame(out)
    return df

In [89]:
#1
rec_num = 15

query = f"""select item, count
                from (  select item, count(user) as count
                        from ratings
                        where rating is not Null
                        group by item) R1
                where item >=150 and item < 350
                order by count desc, item asc"""

sample = get_output(query)
df = pd.DataFrame(sample, columns=['item', 'count'])


In [90]:
with open('test1.txt', 'w') as f:
        f.write(tabulate(df, headers=df.columns, tablefmt='psql', showindex=False))
print("Output printed in test1.txt")

Output printed in test.txt


In [91]:
#2
query = f"""select R3.item, round(avg(R3.P_rating),4) as prediction
                from (
                    select R1.item, R1.user, round((R1.rating - R2.min_rating)/(R2.max_rating-R2.min_rating),4) as P_rating  
                    from (select user, item, rating
                        from ratings
                        where user is Not Null and rating is Not Null) as R1
                        left outer join
                        (select user, max(rating) as max_rating, min(rating) as min_rating
                        from ratings
                        where user is Not Null and rating is Not Null
                        group by user) as R2
                        on R1.user = R2.user
                ) R3
                where R3.item >=150 and R3.item < 350
                group by R3.item
                order by prediction desc, item asc"""

sample = get_output(query)
df = pd.DataFrame(sample, columns=['item', 'prediction'])

print(df)

     item prediction
0     316     0.8666
1     203     0.8634
2     307     0.8347
3     290     0.8321
4     310     0.8311
..    ...        ...
195   151     0.4378
196   271     0.3591
197   264     0.3576
198   285     0.3413
199   269     0.2810

[200 rows x 2 columns]


In [92]:
with open('test2.txt', 'w') as f:
        f.write(tabulate(df, headers=df.columns, tablefmt='psql', showindex=False))
print("Output printed in test2.txt")

Output printed in test.txt


In [88]:
#3

query = f"""select G.user as user, G.item as item, G.prediction as prediction 
            from (select F.user as user, F.item_1 as item, round(sum(F.sim_adj * F.rating_adj), 4) as prediction 
                from (select D.item_1, D.item_2, D.sim_adj, E.user, E.rating_adj 
                from (select B.item_1, B.item_2, round(B.sim / B.sim_sum, 4) as sim_adj
                    from (select A.item_1, A.item_2, A.sim, sum(A.sim) over (partition by A.item_1) as sim_sum 
                    from (select item_1, item_2, sim, row_number() over (partition by item_1 order by sim desc, item_2 asc) as rn from item_similarity) as A 
                    where A.rn <= 5) as B) as D
                left join (
                    select C.item, C.user, case when rating is not null then C.rating else C.avg_rating end as rating_adj from (
                    select item, user, rating, avg(rating) over (partition by item) as avg_rating from ratings) as C
                    ) as E
                on D.item_1 = E.item) as F
                group by F.user, F.item_1
                having F.user = {user} ) as G
            where G.item not in (select item from ratings where user = {user} and rating is not null )
            order by prediction desc, item asc limit {rec_num}"""
sample = get_output(query)
print(sample)

   user  item prediction
0     0   316     4.4839
1     0   203     4.4609
2     0    69     4.4362
3     0   439     4.3889
4     0   157     4.3381


In [51]:
with open('test3.txt', 'w') as f:
        f.write(tabulate(df, headers=df.columns, tablefmt='psql', showindex=False))
print("Output printed in test3.txt")

Output printed in pbr.txt
